# Análise Exploratória de Dados

## Preparação

In [64]:
import polars as pl

from df_utils import get_list_column_max_len, one_hot_encode_list_column
from geo_location import extract_polygons_from_folder, check_point_in_kml_polygon

## Leitura de dados

Os dados se encontram no formato NDJSON, também conhecido como JSONLines,
em que há um objeto JSON por linha no arquivo.

In [6]:
neighborhoods = ["cidade_baixa", "centro_historico", "menino_deus", "sarandi"]

neighborhoods_df_map = {neighborhood: pl.scan_ndjson(f"data/{neighborhood}_listings.json") for neighborhood in neighborhoods}

# raw_menino_deus_df = pl.read_ndjson("data/menino_deus_listings.json")
# raw_cidade_baixa_df = pl.read_ndjson("data/cidade_baixa_listings.json")
# raw_centro_historico_df = pl.read_ndjson("data/centro_historico_listings.json")
# raw_sarandi_df = pl.read_ndjson("data/sarandi_listings.json")

### Entendimento dos dados

In [7]:
neighborhoods_df_map["cidade_baixa"].head(2).collect()

listing,account,medias,accountLink,link
struct[43],struct[9],list[struct[3]],struct[4],struct[4]
"{""dfadbe19-2554-5480-56eb-0780d0ee4019"",[1],[1],""UNIT"",false,[48],""26349587"",[""DISABLED_ACCESS"", ""INTEGRATED_ENVIRONMENTS"", … ""PARTY_HALL""],"""",""2019-11-05T13:09:45.557+00:00"",""ConstructionStatus_NONE"",[8],""NonActivationReason_NONE"",{""Porto Alegre"",""Cidade Baixa"",""Rua Joaquim Nabuco"",null,{""GOOGLE"",-30.041,-51.219,140,null,null},""RS""},[],0,"""",true,""ACTIVE"",""2024-12-17T11:12:11.169+00:00"",[0],[46],""STREET"",null,""USED"",""Apartamento em excelente estado, iluminado e silencioso, mobiliado com móveis embutidos , conceito aberto, cozinha e sala integrados, gás central, aquecimento em todas as torneiras, fica ar condicionado do quarto e todos os móveis embutidos quarto, banheiro, sala, cozinha e homeoffice."",[],""OWNER"",[""RESIDENTIAL""],0,[],""PREMIUM"",""Apartamento de 46 metros quadrados no bairro Cidade Baixa com 1 quarto"",""2463567709"",""a5adbade-589d-3c61-97b7-e4014f21cf3d"",4,[],{"""",[""5130130100""]},[{""SALE"",600,290000,300,null}],true,""GRUPOZAP"",""ZAP1593841"",[""APARTMENT""]}","{""dfadbe19-2554-5480-56eb-0780d0ee4019"",""BÁRBARA EINSFELD DE BORBA"",null,"""",true,345436,3136761,""2018-05-11T05:14:21Z"",""""}","[{""7cb4b1098bf75603a7f2561456bc2883"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/7cb4b1098bf75603a7f2561456bc2883.webp"",""IMAGE""}, {""3ad45448360d515b4bbb3d833537b5e4"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/3ad45448360d515b4bbb3d833537b5e4.webp"",""IMAGE""}, … {""3c74183638571539a2c2f9f568a72d7a"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/3c74183638571539a2c2f9f568a72d7a.webp"",""IMAGE""}]","{""BÁRBARA EINSFELD DE BORBA"",""/imobiliaria/345436/"","""",{}}","{""Apartamento com 1 Quarto à venda, 46m²"",""/imovel/venda-apartamento-1-quarto-mobiliado-cidade-baixa-porto-alegre-46m2-id-2463567709/"","""",{""Porto Alegre"",""Cidade Baixa"","""",""Rua Joaquim Nabuco"","""",""""}}"
"{""90879777-b9cc-65b9-28c8-c86f00947af9"",[1],[2],""UNIT"",false,[66],"""",[],""13887"",""2024-12-01T21:22:02.001+00:00"",""ConstructionStatus_NONE"",[],""NonActivationReason_NONE"",{""Porto Alegre"",""Cidade Baixa"",""Rua José do Patrocínio"",""357"",{""GOOGLE"",null,null,null,-30.038562,-51.224749},""RS""},[0],0,""51999671489"",false,""ACTIVE"",""2024-12-26T12:00:41.422+00:00"",[],[56],""ALL"",7,""USED"",""Compre apartamento silencioso,55,10m2 de área útil,2 quartos, piso parquet,living 2 ambientes,banheiro social ,cozinha e área de serviço. O condomínio fica localizado em Rua José do Patrocínio no bairro Cidade Baixa em Porto Alegre. Está bem situado, próximo a pontos de interesse de Cidade Baixa, tais como Azambuja, Praça General Daltro Filho, Faculdade de Direito da Funda. Escola Sup. do Ministério Público, Praça Salvador Allende, e Policlínica Militar de Porto Alegre.Aceitamos Fiinanciamento Bancário e Fgts.Marque hoje mesmo uma visita com um de nossos Consultores Imobiliários caddstrados! ]]>"",[],""REAL_ESTATE"",[""RESIDENTIAL""],0,[],""PREMIUM"",""Compre apartamento silencioso,55,10m2 de área útil,2 quartos, piso parquet,living 2 ambientes,banhei"",""2760694515"",""40017671-36da-3cb2-98e7-116dd4abcba1"",0,[],{"""",[""5132084035"", ""51999671489""]},[{""SALE"",0,275000,290,null}],true,""GRUPOZAP"",""VR527903"",[""APARTMENT""]}","{""90879777-b9cc-65b9-28c8-c86f00947af9"",""Sperinde Vendas"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/fb5adab24465e4ba59ba7d6e4ac1de9e.webp"",""00411-J-RS"",true,33518,2594637,""2018-03-27T18:49:45Z"",""diamond""}","[{""3d8f346437754cf315cc942816497178"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/3d8f346437754cf315cc942816497178.webp"",""IMAGE""}, {""d309f06d26092646e716b47e93936c7d"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/d309f06d26092646e716b4

#### Qual o tamanho da massa de dados?

In [8]:
total = 0
for neighborhood, df in neighborhoods_df_map.items():
    df_height = df.select(pl.len()).collect().item()
    print(f"{neighborhood}: {df_height}")
    total += df_height
print(f"Total: {total}")

cidade_baixa: 2790
centro_historico: 6007
menino_deus: 5833
sarandi: 2711
Total: 17341


In [9]:
columns = neighborhoods_df_map["cidade_baixa"].collect_schema().names()
print(f'Temos {len(columns)} colunas: {", ".join(columns)}.')

Temos 5 colunas: listing, account, medias, accountLink, link.


#### Pré-processamento inicial - `listing`

Todas as informações que consideramos importantes para nossa análise estão na coluna
`listing`, que tem uma estrutura aninhada complexa.
Aplainamos a estrutura para simplificar a análise.

In [10]:
listings_df_map = {
    neighborhood: df.select(pl.col("listing").struct.unnest())
    for neighborhood, df in neighborhoods_df_map.items()
}

In [11]:
listings_df_map["cidade_baixa"].head(3).collect()

advertiserId,bathrooms,bedrooms,propertyType,resale,totalAreas,legacyId,amenities,providerId,createdAt,constructionStatus,floors,nonActivationReason,address,suites,unitFloor,whatsappNumber,acceptExchange,status,updatedAt,capacityLimit,usableAreas,displayAddressType,listingsCount,listingType,description,parkingSpaces,contractType,usageTypes,buildings,stamps,publicationType,title,id,sourceId,unitsOnTheFloor,unitSubTypes,advertiserContact,pricingInfos,showPrice,portal,externalId,unitTypes
str,list[i64],list[i64],str,bool,list[i64],str,list[str],str,str,str,list[i64],str,struct[6],list[i64],i64,str,bool,str,str,list[i64],list[i64],str,i64,str,str,list[i64],str,list[str],i64,list[str],str,str,str,str,i64,list[str],struct[2],list[struct[5]],bool,str,str,list[str]
"""dfadbe19-2554-5480-56eb-0780d0…",[1],[1],"""UNIT""",false,[48],"""26349587""","[""DISABLED_ACCESS"", ""INTEGRATED_ENVIRONMENTS"", … ""PARTY_HALL""]","""""","""2019-11-05T13:09:45.557+00:00""","""ConstructionStatus_NONE""",[8],"""NonActivationReason_NONE""","{""Porto Alegre"",""Cidade Baixa"",""Rua Joaquim Nabuco"",null,{""GOOGLE"",-30.041,-51.219,140,null,null},""RS""}",[],0,"""""",true,"""ACTIVE""","""2024-12-17T11:12:11.169+00:00""",[0],[46],"""STREET""",null,"""USED""","""Apartamento em excelente estad…",[],"""OWNER""","[""RESIDENTIAL""]",0,[],"""PREMIUM""","""Apartamento de 46 metros quadr…","""2463567709""","""a5adbade-589d-3c61-97b7-e4014f…",4,[],"{"""",[""5130130100""]}","[{""SALE"",600,290000,300,null}]",true,"""GRUPOZAP""","""ZAP1593841""","[""APARTMENT""]"
"""90879777-b9cc-65b9-28c8-c86f00…",[1],[2],"""UNIT""",false,[66],"""""",[],"""13887""","""2024-12-01T21:22:02.001+00:00""","""ConstructionStatus_NONE""",[],"""NonActivationReason_NONE""","{""Porto Alegre"",""Cidade Baixa"",""Rua José do Patrocínio"",""357"",{""GOOGLE"",null,null,null,-30.038562,-51.224749},""RS""}",[0],0,"""51999671489""",false,"""ACTIVE""","""2024-12-26T12:00:41.422+00:00""",[],[56],"""ALL""",7,"""USED""","""Compre apartamento silencioso,…",[],"""REAL_ESTATE""","[""RESIDENTIAL""]",0,[],"""PREMIUM""","""Compre apartamento silencioso,…","""2760694515""","""40017671-36da-3cb2-98e7-116dd4…",0,[],"{"""",[""5132084035"", ""51999671489""]}","[{""SALE"",0,275000,290,null}]",true,"""GRUPOZAP""","""VR527903""","[""APARTMENT""]"
"""a70a64d3-0e45-b5bc-5b75-0ed9fc…",[1],[1],"""UNIT""",false,[56],"""""","[""PETS_ALLOWED"", ""FURNISHED""]","""63639""","""2024-12-19T05:29:55.171+00:00""","""ConstructionStatus_NONE""",[],"""NonActivationReason_NONE""","{""Porto Alegre"",""Centro Histórico"",""Avenida João Pessoa"",""371"",{""GOOGLE"",null,null,null,-30.035694,-51.221204},""RS""}",[],3,"""51998432762""",false,"""ACTIVE""","""2024-12-26T18:08:02.605+00:00""",[],[42],"""ALL""",null,"""USED""","""Ótimo apartamento à venda no E…",[],"""REAL_ESTATE""","[""RESIDENTIAL""]",0,[],"""STANDARD""","""Apartamento à venda em Porto A…","""2765177708""","""38efd784-97ce-3b42-a7a2-2bb74b…",0,[],"{"""",[""51998432762""]}","[{""SALE"",335,175000,530,null}]",true,"""GRUPOZAP""","""460784""","[""APARTMENT""]"


#### Descarte de informações não importantes

Existe muita informação ruidosa nesses dados.
Escolhemos as seguintes colunas para descarte.

In [12]:
COLUMNS_TO_DROP = [
    "advertiserId",
    "constructionStatus",
    "portal",
    "stamps",
    "advertiserContact",
    "whatsappNumber",
    "title",
    "nonActivationReason",
    "status",
    "legacyId",
    "externalId",
    "listingsCount",
    "createdAt",
    "updatedAt",
    "showPrice",
    "acceptExchange",
    "description",
    "sourceId",
    "providerId"
]

In [13]:
listings_df_map = {
    name: df.drop(COLUMNS_TO_DROP)
    for name, df in listings_df_map.items()
}

In [14]:
listings_df_map["cidade_baixa"].head(3).collect()

bathrooms,bedrooms,propertyType,resale,totalAreas,amenities,floors,address,suites,unitFloor,capacityLimit,usableAreas,displayAddressType,listingType,parkingSpaces,contractType,usageTypes,buildings,publicationType,id,unitsOnTheFloor,unitSubTypes,pricingInfos,unitTypes
list[i64],list[i64],str,bool,list[i64],list[str],list[i64],struct[6],list[i64],i64,list[i64],list[i64],str,str,list[i64],str,list[str],i64,str,str,i64,list[str],list[struct[5]],list[str]
[1],[1],"""UNIT""",false,[48],"[""DISABLED_ACCESS"", ""INTEGRATED_ENVIRONMENTS"", … ""PARTY_HALL""]",[8],"{""Porto Alegre"",""Cidade Baixa"",""Rua Joaquim Nabuco"",null,{""GOOGLE"",-30.041,-51.219,140,null,null},""RS""}",[],0,[0],[46],"""STREET""","""USED""",[],"""OWNER""","[""RESIDENTIAL""]",0,"""PREMIUM""","""2463567709""",4,[],"[{""SALE"",600,290000,300,null}]","[""APARTMENT""]"
[1],[2],"""UNIT""",false,[66],[],[],"{""Porto Alegre"",""Cidade Baixa"",""Rua José do Patrocínio"",""357"",{""GOOGLE"",null,null,null,-30.038562,-51.224749},""RS""}",[0],0,[],[56],"""ALL""","""USED""",[],"""REAL_ESTATE""","[""RESIDENTIAL""]",0,"""PREMIUM""","""2760694515""",0,[],"[{""SALE"",0,275000,290,null}]","[""APARTMENT""]"
[1],[1],"""UNIT""",false,[56],"[""PETS_ALLOWED"", ""FURNISHED""]",[],"{""Porto Alegre"",""Centro Histórico"",""Avenida João Pessoa"",""371"",{""GOOGLE"",null,null,null,-30.035694,-51.221204},""RS""}",[],3,[],[42],"""ALL""","""USED""",[],"""REAL_ESTATE""","[""RESIDENTIAL""]",0,"""STANDARD""","""2765177708""",0,[],"[{""SALE"",335,175000,530,null}]","[""APARTMENT""]"


#### Atributos em forma de lista

Sabemos que `amenities` descrevem as amenidades presentes em um dado imóvel,
o que pode ser interessante transformar em atributos preditores para nossas regressões.

Entretanto, ainda precisamos entender outras colunas que tem listas como valores.
Várias delas não parecem fazer sentido em ter multiplicidade: em que contexto o número
de quartos (bedrooms) precisa ser uma lista? Número de banheiros? Área utilizável?
Uma unidade pode pertencer ao mesmo tempo a mais de um tipo (comercial, casa)?

Em primeiro lugar, tentamos entender quais listas de fato vêm a possuir mais de um
elemento.


In [46]:
LIST_COLUMNS = [
    "floors",
    "unitSubTypes",
    "suites",
    "unitTypes",
    "pricingInfos",
    "parkingSpaces",
    "totalAreas",
    "bathrooms",
    "bedrooms",
    "usableAreas",
    "usageTypes",
    "capacityLimit",
]

max_colname_width = max(len(name) for name in LIST_COLUMNS)
arrays_with_size_larger_than_one = []
for name, df in listings_df_map.items():
    print(name)
    print("=" * len(name))
    for colname in LIST_COLUMNS:
        column_max_length = get_list_column_max_len(df, colname)
        print(f"{colname:<{max_colname_width}}: {column_max_length}")
        if column_max_length > 1:
            arrays_with_size_larger_than_one.append(colname)
    print("\n")

cidade_baixa
floors       : 1
unitSubTypes : 2
suites       : 1
unitTypes    : 1
pricingInfos : 2
parkingSpaces: 2
totalAreas   : 2
bathrooms    : 2
bedrooms     : 2
usableAreas  : 2
usageTypes   : 2
capacityLimit: 1


centro_historico
floors       : 1
unitSubTypes : 2
suites       : 1
unitTypes    : 1
pricingInfos : 2
parkingSpaces: 2
totalAreas   : 2
bathrooms    : 2
bedrooms     : 2
usableAreas  : 2
usageTypes   : 2
capacityLimit: 1


menino_deus
floors       : 1
unitSubTypes : 2
suites       : 1
unitTypes    : 1
pricingInfos : 2
parkingSpaces: 2
totalAreas   : 2
bathrooms    : 2
bedrooms     : 2
usableAreas  : 2
usageTypes   : 2
capacityLimit: 1


sarandi
floors       : 1
unitSubTypes : 2
suites       : 1
unitTypes    : 1
pricingInfos : 2
parkingSpaces: 2
totalAreas   : 2
bathrooms    : 2
bedrooms     : 2
usableAreas  : 2
usageTypes   : 2
capacityLimit: 1




Para entender melhor esses atributos, filtramos nossos dados para as situações em que
essas multiplicidades ocorrem.

In [63]:
listings_df_map["centro_historico"].filter(
    pl.col("unitSubTypes").list.len() > 1
).collect().head(3)

bathrooms,bedrooms,propertyType,resale,totalAreas,amenities,floors,address,suites,unitFloor,capacityLimit,usableAreas,displayAddressType,listingType,parkingSpaces,contractType,usageTypes,buildings,publicationType,id,unitsOnTheFloor,unitSubTypes,pricingInfos,unitTypes
list[i64],list[i64],str,bool,list[i64],list[str],list[i64],struct[6],list[i64],i64,list[null],list[i64],str,str,list[i64],str,list[str],i64,str,str,i64,list[str],list[struct[5]],list[str]
[3],[3],"""UNIT""",false,[253],"[""BICYCLES_PLACE"", ""ELEVATOR"", … ""PETS_ALLOWED""]",[8],"{""Porto Alegre"",""Centro Histórico"",""Rua Avaí"",""40"",{""GOOGLE"",null,null,null,-30.034,-51.222745},""RS""}",[1],8,[],[192],"""ALL""","""USED""",[1],"""REAL_ESTATE""","[""RESIDENTIAL""]",1,"""STANDARD""","""2685510440""",2,"[""PENTHOUSE"", ""DUPLEX""]","[{""SALE"",3400,975000,1200,null}]","[""APARTMENT""]"
[2],[3],"""UNIT""",false,[184],"[""PARTY_HALL"", ""BALCONY"", … ""ELEVATOR""]",[9],"{""Porto Alegre"",""Centro Histórico"",""Avenida João Pessoa"",""721"",{""GOOGLE"",null,null,null,-30.038406,-51.219321},""RS""}",[1],0,[],[133],"""ALL""","""USED""",[1],"""REAL_ESTATE""","[""RESIDENTIAL""]",0,"""STANDARD""","""2761198831""",4,"[""PENTHOUSE"", ""DUPLEX""]","[{""SALE"",2400,660000,909,null}]","[""APARTMENT""]"


`unitSubTypes`, neste caso, define duas categorias que se aplicam ao imóvel:
ser uma cobertura, e ser um duplex. Faz sentido transformar os valores desta coluna
em possíveis preditores para nossas regressões.

In [62]:
listings_df_map["centro_historico"].filter(
    pl.col("pricingInfos").list.len() > 1
).collect().head(3)

bathrooms,bedrooms,propertyType,resale,totalAreas,amenities,floors,address,suites,unitFloor,capacityLimit,usableAreas,displayAddressType,listingType,parkingSpaces,contractType,usageTypes,buildings,publicationType,id,unitsOnTheFloor,unitSubTypes,pricingInfos,unitTypes
list[i64],list[i64],str,bool,list[i64],list[str],list[i64],struct[6],list[i64],i64,list[null],list[i64],str,str,list[i64],str,list[str],i64,str,str,i64,list[str],list[struct[5]],list[str]
[1],[0],"""UNIT""",false,[42],[],[],"{""Porto Alegre"",""Centro Histórico"",""Rua dos Andradas"",""1137"",{""GOOGLE"",null,null,null,-30.030361,-51.230258},""RS""}",[0],0,[],[42],"""ALL""","""USED""",[0],"""REAL_ESTATE""","[""COMMERCIAL""]",0,"""PREMIUM""","""2712201134""",0,[],"[{""RENTAL"",1289,1200,500,{""MONTHLY"",[],1700}}, {""SALE"",1289,160000,500,null}]","[""OFFICE""]"
[1],[0],"""UNIT""",false,[35],[],[],"{""Porto Alegre"",""Cidade Baixa"",""Avenida Loureiro da Silva"",""2001"",{""GOOGLE"",null,null,null,-30.035203,-51.221879},""RS""}",[0],0,[],[35],"""ALL""","""USED""",[2],"""REAL_ESTATE""","[""COMMERCIAL""]",0,"""STANDARD""","""2570524388""",0,[],"[{""RENTAL"",331,1350,739,{""MONTHLY"",[],2089}}, {""SALE"",331,265000,739,null}]","[""OFFICE""]"
[2],[0],"""UNIT""",false,[65],"[""ELEVATOR""]",[22],"{""Porto Alegre"",""Centro Histórico"",""Avenida Senador Salgado Filho"",null,{""GOOGLE"",-30.031,-51.224,140,null,null},""RS""}",[0],0,[],[52],"""STREET""","""USED""",[1],"""REAL_ESTATE""","[""COMMERCIAL""]",0,"""STANDARD""","""2743763450""",0,[],"[{""RENTAL"",150,1100,160,{""MONTHLY"",[],1260}}, {""SALE"",150,275000,160,null}]","[""OFFICE""]"


`pricingInfos` tem multiplicidade quando é possível comprar _ou_ alugar um imóvel.
Para nossos propósitos, nos interessamos tão somente pelo valor de compra.

Existem dados duplicados?

In [39]:
for name, df in listings_df_map.items():
    num_dupes = df.filter(pl.col("id").is_duplicated()).collect().height
    print(f"{name}: {num_dupes} duplicatas")

cidade_baixa: 102 duplicatas
centro_historico: 712 duplicatas
menino_deus: 904 duplicatas
sarandi: 132 duplicatas


In [37]:
neighborhoods_df_map["cidade_baixa"].filter(pl.col("listing").struct.field("id") == "2761415335").collect()

listing,account,medias,accountLink,link
struct[43],struct[9],list[struct[3]],struct[4],struct[4]
"{""10929a8c-7227-5aff-fb05-6bf3945bf5b7"",[1],[2],""UNIT"",false,[104],"""",[],""83977"",""2024-12-04T20:20:07.087+00:00"",""ConstructionStatus_NONE"",[],""NonActivationReason_NONE"",{""Porto Alegre"",""Farroupilha"",""Avenida Venâncio Aires"",""449"",{""GOOGLE"",null,null,null,-30.041217,-51.215602},""RS""},[0],0,""51992941900"",false,""ACTIVE"",""2024-12-25T20:16:09.176+00:00"",[],[82],""ALL"",7,""USED"",""Excelente oportunidade no coração de Porto Alegre!<BR><BR>Este apartamento encantador de 81m² de área privativa está localizado na Avenida Venâncio Aires, em uma região privilegiada, próximo à Avenida João Pessoa, Parque Farroupilha (Redenção) e ao bairro Bom Fim.<BR><BR>Destaques do imóvel:<BR>02 dormitórios espaçosos;<BR>Amplo living com 02 ambientes (estar e jantar);<BR>Cozinha separada e área de serviço ampliada;<BR>Piso em parquet, que adiciona charme e conforto aos ambientes;<BR>Imóvel semimobiliado, bem ventilado, ensolarado e reformado.<BR><BR>Localização estratégica:<BR>Fácil acesso aos principais serviços, comércio e lazer dos bairros Cidade Baixa, Santana e Bom Fim.<BR><BR>Ideal para quem busca conforto, praticidade e qualidade de vida!<BR><BR>Entre em contato e agende sua visita. Não perca essa oportunidade de viver bem em Porto Alegre! ]]>"",[],""REAL_ESTATE"",[""RESIDENTIAL""],0,[],""STANDARD"",""Excelente oportunidade no coração de Porto Alegre!Este apartamento encantador de 81m² de área privat"",""2761415335"",""c6773be8-271d-36e2-95b9-07d1d405165c"",0,[],{"""",[""5137373337"", ""51992941900""]},[{""SALE"",0,430000,550,null}],true,""GRUPOZAP"",""BE11051"",[""APARTMENT""]}","{""10929a8c-7227-5aff-fb05-6bf3945bf5b7"",""Orientta Negócios Imobiliários"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/700237e1377fef8be63496af3d015bcc.webp"",""36071-F-RS"",true,54505,2594899,""2018-03-27T16:55:54Z"",""gold""}","[{""02cb37650ba179846553f62e62610c71"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/02cb37650ba179846553f62e62610c71.webp"",""IMAGE""}, {""a6593237d8c622c272b7149661d39ffc"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/a6593237d8c622c272b7149661d39ffc.webp"",""IMAGE""}, … {""ae5c0129d8206466ddaa81ed822f9c90"",""https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/ae5c0129d8206466ddaa81ed822f9c90.webp"",""IMAGE""}]","{""Orientta Negócios Imobiliários"",""/imobiliaria/54505/"","""",{}}","{""Apartamento com 2 Quartos à venda, 82m²"",""/imovel/venda-apartamento-2-quartos-farroupilha-porto-alegre-82m2-id-2761415335/"","""",{""Porto Alegre"",""Farroupilha"","""",""Avenida Venâncio Aires"",""449"",""""}}"
"{""10929a8c-7227-5aff-fb05-6bf3945bf5b7"",[1],[2],""UNIT"",false,[102],"""",[],""83977"",""2024-12-04T20:20:07.087+00:00"",""ConstructionStatus_NONE"",[],""NonActivationReason_NONE"",{""Porto Alegre"",""Farroupilha"",""Avenida Venâncio Aires"",""449"",{""GOOGLE"",null,null,null,-30.041217,-51.215602},""RS""},[0],0,""51992941900"",false,""ACTIVE"",""2024-12-26T20:27:47.915+00:00"",[],[80],""ALL"",2,""USED"",""Excelente oportunidade no coração de Porto Alegre!<BR><BR>Este apartamento encantador de 81m² de área privativa está localizado na Avenida Venâncio Aires, em uma região privilegiada, próximo à Avenida João Pessoa, Parque Farroupilha (Redenção) e ao bairro Bom Fim.<BR><BR>Destaques do imóvel:<BR>02 dormitórios espaçosos;<BR>Amplo living com 02 ambientes (estar e jantar);<BR>Cozinha separada e área de serviço ampliada;<BR>Piso em parquet, que adiciona charme e conforto aos ambientes;<BR>Imóvel semimobiliado, bem ventilado, ensolarado e reformado.<BR><BR>Localização estratégica:<BR>Fácil acesso aos principais serviços, comércio e lazer dos bairros Cidade Baixa, Santana e Bom Fim.<BR><BR>Ideal para quem busca conforto, praticidade e qualidade de vida!<BR><BR>Entre em contato e agende sua visita

In [ ]:
kml_data_file = "data/cheias_em_porto_alegre.kml"
polygons = extract_polygons_from_folder(
    kml_data_file, "Inundação simulada com nível 500 cm (5.0 m)"
)